In [1]:
import pandas as pd
import numpy as np
import keras
from gensim.models import word2vec
from pymongo import MongoClient

Using TensorFlow backend.


In [2]:
from itertools import chain
import re
from collections import Counter
import math
from random import randint
import jieba

## 拿data

In [5]:
def get_data(database,collection):
    
    """ 連接DB 
    """    
    client = MongoClient('client')
    db = client[database]
    collection = db[collection]
    print(collection.stats)
    """ 拿到data
    """
    cursor = collection.find()
    data = pd.DataFrame(list(cursor))
 
    print('finish')
    
    return data

In [6]:
""" article: _id, title, keyword
    test: _id, content, cate, recList
"""
raw_data = get_data('foodnext','article')
raw_data2 = get_data('foodnext','test')
data = pd.merge(raw_data,raw_data2,how='left',on='_id')
data = data.dropna()
data = data.drop(['keyword','cate'],axis=1)
data.head()

Collection(Database(MongoClient(host=['r.xnet.world:27017'], document_class=dict, tz_aware=False, connect=True), 'foodnext'), 'article.stats')
finish
Collection(Database(MongoClient(host=['r.xnet.world:27017'], document_class=dict, tz_aware=False, connect=True), 'foodnext'), 'test.stats')
finish


,_id,title,content,description,recList
2,/cloud/consume/paper/3098764222,「食品冒充」與媒體的「庶民偽裝」 @ 食力foodNEXT‧食事求實的知識頻道,作者＝柏井壽繼連鎖飯店的附屬餐廳公開道歉後，說得誇張一點，幾乎全日本主要的飯店與餐廳都在連續...,繼連鎖飯店的附屬餐廳公開道歉後，說得誇張一點，幾乎全日本主要的飯店與餐廳都在連續舉辦記者會道...,"[/science/machining/paper/5739111556, /news/ne..."
3,/cloud/consume/paper/3098765022,為什麼小小花生米，連續18年抽驗都有事？ @ 食力foodNEXT‧食事求實的知識頻道,撰文=林正文食藥署公布2015年度市售花生製品黃麴毒素檢驗結果，共抽驗218件產品，有七件花...,食藥署公布104年度市售花生製品黃麴毒素檢驗結果，共抽驗218件產品，有七件花生製品的黃麴毒...,"[/news/newsnow/paper/4616124369, /news/newsfal..."
4,/cloud/consume/paper/3098767727,台灣麵包職人開店經驗分享 @ 食力foodNEXT‧食事求實的知識頻道,作者=田川美由a.開麵包店需要申請哪些文件？需要遵守哪些規定？台灣目前對於麵包店的法規並沒有...,台灣目前對於麵包店的法規並沒有像國外那麼嚴格，但如果有一定的銷售量、想要長期經營，建議至少要...,"[/life/placemaking/paper/5852303844, /life/pla..."
5,/cloud/consume/paper/3098768327,想喝路邊攤的現榨果汁？小心細菌一起喝下肚！！！ @ 食力foodNEXT‧食事求實的知識頻道,撰文＝蕭琮容逛市集與夜市的時候，果汁攤販常常是生意最好的那攤。台灣是水果王國，各式各樣的組合...,路邊的果汁看來是路邊攤相對天然的選擇，但每年各地衛生局的抽檢結果可以發現，路邊的現榨果汁非常...,"[/news/newsnation/paper/4357961898, /news/news..."
6,/cloud/consume/paper/3098768527,市面真假蜂蜜充斥，鑑定方法有用嗎？ @ 食力foodNEXT‧食事求實的知識頻道,撰文＝蕭琮容2015年11月中旬台北市衛生局抽驗市面蜂蜜，赫然發現不少假的「純蜜」商品，來源...,市面真假蜜充斥，辨別方法眾說紛紜。到底這些說法的科學依據是什麼呢？一般消費者又該如何在缺乏儀...,"[/news/newstrack/paper/3357915296, /science/te..."


## 處理title和content

In [7]:
def jieba_tokenizer(text):

    words = jieba.cut(text)

    return ' '.join(words)

In [8]:
data['title'] = data['title'].apply(lambda x: x.strip('@ 食力foodNEXT‧食事求實的知識頻道'))
data['title_tokenized'] = data.loc[:,'title'].apply(jieba_tokenizer)
data['title_tokenized'] = data.loc[:,'title_tokenized'].apply(lambda x: re.sub('[#！、，「」（）。《》()【】：？－]','',x))

data['content_tokenized'] = data.loc[:,'content'].apply(jieba_tokenizer)
data['content_tokenized'] = data.loc[:,'content_tokenized'].apply(lambda x: re.sub('[#！＝=、，「」（）。《》()【】：？－]','',x))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Jason\AppData\Local\Temp\jieba.cache
Loading model cost 1.610 seconds.
Prefix dict has been built succesfully.


## 加入廣告

In [9]:
""" 拿data
"""
product = get_data('product', 'product')

Collection(Database(MongoClient(host=['r.xnet.world:27017'], document_class=dict, tz_aware=False, connect=True), 'product'), 'product.stats')
finish


In [10]:
""" 處理 description
"""
product['des_tokenized'] = product.loc[:,'description'].apply(jieba_tokenizer)
product['des_tokenized'] = product.loc[:,'des_tokenized'].apply(lambda x: re.sub('[#！、，「」（）。《》()【】：？－～]','',x))


In [11]:
""" 處理 item name
"""
product['name_tokenized'] = product.loc[:,'itemName'].apply(jieba_tokenizer)
product['name_tokenized'] = product.loc[:,'name_tokenized'].apply(lambda x: re.sub('[#！、，「」（）。《》()【】：？－～]','',x))

## word2vec model

In [12]:
corpus = data['content_tokenized'].values.tolist()
corpus_list = list(map(lambda x : x.split(), corpus))

In [13]:
""" 用corpus建立模型
"""
# model = word2vec.Word2Vec(corpus_list,size=300,iter=10,sg=1,hs=0)
# model.save('word2vec.model')

' 用corpus建立模型\n'

In [14]:
""" 將廣告加入原本的corpus建立模型
"""
description = product['des_tokenized'].values.tolist()
description_list = list(map(lambda x: x.split(), description))
corpus_list += description_list

In [ ]:
model = word2vec.Word2Vec(corpus_list,size=300,iter=10,sg=1,hs=0)
model.save('word2vec_with_ad.model')

## 加入權重 $ weight = \cos \times count$  

In [15]:
def corpus(number):
    
    print(rec.iloc[number,:])
    return rec['top10'].values[number]

In [16]:
def recomm(w2v_model, words, topn=5):
    similar_df = pd.DataFrame()
    word_list = []

    """ 用模型產生出相關詞 list
    """
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=['word', 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=0)
             
        except:
            print(word, "not found in Word2Vec model!")
    """ 判斷相關詞有沒有在 top10 tags裡面
        如果有, 便返回其 id與 title
    """
    recomm = []
    counter = []
    cos =[]
    word_list = similar_df['word'].values
    cos_list = similar_df['cos'].values

    for i in range(rec.shape[0]):
        for j in range(len(word_list)):  
            if word_list[j] in rec.top10[i]:
                tmp = rec.pageid[i]
                tmp2 = cos_list[j]

                recomm.append(tmp)
                cos.append(tmp2)
    
    recomm_df = pd.DataFrame({'_id': recomm,'cos': cos})
    recomm_df = pd.merge(recomm_df,data,how='left',on='_id')
    recomm_df = recomm_df.iloc[:,0:3]
    recomm_df['count'] = 1
    recomm_df = recomm_df.groupby(['_id','title','cos']).sum().reset_index()
    recomm_df['weight'] = recomm_df['cos'] * recomm_df['count']
    recomm_df = recomm_df.groupby(['_id','title'])[['weight']].sum().sort_values(['weight'],ascending=False).reset_index()
    filter = recomm_df['weight'] < recomm_df.weight.values[0]
    recomm_df = recomm_df[filter]
    recomm_df = recomm_df.drop_duplicates('weight', 'first', inplace=False)
    recomm_df_title = list(set(recomm_df.title))
 
    return recomm_df,recomm_df_title

""" 產生推薦廣告
"""
def recomm_ad(w2v_model, words, topn=3):
    similar_df = pd.DataFrame()
    word_list = []
    
    """ 用模型產生出相關詞 list
    """
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
            word_list.append(similar_words[word].values.tolist())
             
        except:
            print(word, "not found in Word2Vec model!")
    
    word_list = list(set(list(chain(*word_list))))
    
    """ 判斷相關詞有沒有在廣告的名稱內, 
        如果有, 便返回其 id與 title
    """
    recomm_ad = []
    for word in word_list:
        for i in range(product.shape[0]):

            if word in product.name_tokenized[i]:
                tmp = product._id[i]
                recomm_ad.append(tmp)
                
    recomm_ad_df = pd.DataFrame(recomm_ad, columns=['_id'])
    recomm_ad_df = pd.merge(recomm_ad_df,product,how='left',on='_id')
    recomm_ad_df = recomm_ad_df.loc[:,['_id','itemName']]
    recomm_ad_df_title = list(set(recomm_ad_df.itemName))
 
    return recomm_ad_df, recomm_ad_df_title

In [17]:
model = word2vec.Word2Vec.load('word2vec_with_ad.model')

In [18]:
""" rec: 用tf-idf建立的top10 tags
"""
rec = pd.read_json('c:/Users/Jason/Desktop/rec.json',encoding='utf-8')

In [19]:
""" 輸入文章編號 0-6129
"""
words = corpus(0)
""" model: 使用的model
    topn: 要取幾個相關詞 
"""
recomm_df, recomm_df_title = recomm(model, words, topn=5)
recomm_ad_df, recomm_ad_df_title = recomm_ad(model, words, topn=3)

pageid                              /issue/paper/4098724121
top10     [橄欖油, 冷壓, 粕油, 橄欖, 精製, virgin, olive, 特級, oil, 標準]
Name: 0, dtype: object


## 依照上面表單生成推薦的dataframe
* 11篇文章
* 插1篇廣告
* 如果沒有廣告, 便在補1篇文章
* 如果文章不足11篇, 有多少取多少
* _id + title

In [24]:
def make_final_df(recomm_df, recomm_ad_df):
    
    try:
        final_df = pd.DataFrame()
        final_df = pd.concat([final_df, recomm_df.iloc[:,:2].head(11)], axis=0)
        recomm_ad_df = recomm_ad_df.rename(columns= {'itemName':'title'})
        final_df = final_df.append(recomm_ad_df.iloc[0,:])
    
    except:
        final_df = final_df.append(recomm_df.iloc[12,:2])
    
    final_df = final_df.sample(frac=1).reset_index().drop(['index'],axis=1)
    
    return final_df
    

In [26]:
final_df = make_final_df(recomm_df, recomm_ad_df)
final_df

,_id,title
0,https://www.food123.com.tw/site/sku/2029146/%E...,菲律賓冷壓初榨椰子油
1,/news/newsnow/paper/4593778376,擔心買到摻假初榨橄欖油？未來品質鑑定有新方式！
2,/issue/paper/4616127369,如何挑選米？擇米守則解析
3,/issue/paper/4739524052,義大利知名橄欖油因農藥殘留自主下架
4,/science/scsource/paper/5098100228,米裡面有砷，到底該不該擔心？
5,/issue/paper/4616125169,義大利人判別橄欖油靠感官也靠科學
6,/science/machining/paper/4616175364,標榜冷壓橄欖油就一定比較好嗎？這可未必！
7,/news/newssafe/paper/4616114762,你家是一瓶油炒天下嗎？ 油品專家教你選對好油！
8,/news/newsfalse/paper/5975261638,網路上流傳許多芥花油的謠言，是真的嗎？
9,/issue/paper/5111110318,殘留超標的「標」到底是怎麼來的？
